# Compute Pad heuristics for different time windows

Timewindows are defined of a constant length. Pad operations are not assumed to start at the same timestamp, so the timewindows are defined for `pad_timestamp_start + i * window_length` for `i` 0, 1... such that `i * window_length` < pad_timestamp_end for each pad separately.

In [1]:
from analytics import operation_builder, parser, visualization

import config
import os
import pandas as pd
import run_analytics
import sys

#### Read the pads:

In [2]:
# Get the lists of elementary operations
list_of_elem_ops_per_pad = {}
root_of_dbs = "../../Data/private/"
for (dirpath, dirnames, filenames) in os.walk(root_of_dbs):
    for filename in filenames:
        if ".sql" in filename:
            path_to_db = os.path.join(dirpath, filename)
            list_of_elem_ops_per_main = parser.get_elem_ops_per_pad_from_db(
                path_to_db, "sql_dump")
            list_of_elem_ops_per_pad = {**list_of_elem_ops_per_pad,
                                        **list_of_elem_ops_per_main}

# We create the operation from the list of elementary operations
pads, _, _ = operation_builder.build_operations_from_elem_ops(
    list_of_elem_ops_per_pad,
    config.maximum_time_between_elem_ops)

Get some information about starting and ending timestamps for each pad and filter out the ones we are not interested in:

In [3]:
start_times = {pad_name: pad.get_timestamps()[0] for pad_name, pad in pads.items()}
end_times = {pad_name: pad.get_timestamps()[1] for pad_name, pad in pads.items()}
time_lengths = {pn: end_times[pn] - p for pn, p in start_times.items()}

# Filtering for looking at one single pad (for debugging)
# pads = {"edusearch2014_report-328-25-3": pads["edusearch2014_report-328-25-3"]}

# We will ignore these pads in our analysis
# They last longer than 1 day and have one single author
pads_2_days_and_1_author = [pn for pn, p in time_lengths.items()
                            if p > 5*15*60000]
# Filter out the pads that we identified
pads = {pn: p for pn, p in pads.items() if pn not in pads_2_days_and_1_author}

# pads_safety = pads.copy()

# Print some information about starting and ending timestamps
print(min(start_times.values())-1412586498527, max(start_times.values())-1412586498527)
print(min(end_times.values())-1412586498527, max(end_times.values())-1412586498527)
print()
print(min(time_lengths.values()), max(time_lengths.values()))

#### Generate tsv files 

Here I iterate through different time windows, and save one tsv file for each of the time windows.

Each file contains the information for the pads that have some activity during this time windows (if the pads have finished before the current time window, they are not included in the tsv file).

There are two types of heuristics that are computed:
1. For each elementary operation
2. For the entire pad
    1. Computed from start to end
    2. Computed from a specific start time to the end

Here we are generating the files for 2 (entire pad). In this category there are two types: The ones that are computed from start to end and the ones that are computed from a specific start time to the end.

In order to get the heuristics for specific time windows, we need to specify the start and end timestamps:
* End timestamp is specified by using `pad.pad_at_timestamp(end_window)`
* Start timestamp is specified by using `run_analytics.run` with the parameter `start_time=start_times_old`

Therefore:

In [7]:
# pads = pads_safety.copy()

window_length = 60000 * 5 # 5 minutes-long windows
current_minute = 0
pads_copy = pads.copy()

while len(pads):
    # Prepare for printing to file
    orig_stdout = sys.stdout
    f = open('entries_{}.tsv'.format(current_minute), 'w')
    sys.stdout = f
    
    pads_window = {}
    start_times_old = {}

    # Get copies of each pad for the current time window
    for pad_name, pad in pads.items():
        # Increase by 1 minute
        end_window = start_times[pad_name] + window_length - 1
        # Get the pad ending at end_window
        new_pad, new_elem_ops = pad.pad_at_timestamp(end_window)
        # Create the paragraphs
        new_pad.create_paragraphs_from_ops(new_elem_ops,
            ignore_assertions=True)
        # Save these start times for later
        start_times_old[pad_name] = start_times[pad_name]
        # Save this pad in the dictionary
        pads_window[pad_name] = new_pad
        # Update time values
        start_times[pad_name] = end_window + 1

        if end_times[pad_name] < start_times[pad_name] + window_length - 1:
            del pads_copy[pad_name]

    # Compute and save stats for chosen time windows
    run_analytics.run(None,
        generate_csv_summary=True,
        start_time=start_times_old,
        pads=pads_window)

    pads = pads_copy.copy()
    
    # Stop printing to file
    current_minute += 1
    sys.stdout = orig_stdout
    f.close()

You can check the code for the function `run_analytics.run` to see how the parameter `start_times` is used -specifically, it is used by the functions to compute metrics that are defined in `Pad.py`.

As the paragraphs are created, superparagraphs are also created - that is, with function `pad.create_paragraphs_from_ops`. Each paragraph represents either one new line character (`\n`) or one text line that does not contain any new line characters. The concept of superparagraphs is introduced in order to allow groupings of text lines that are separated by only one new line character, and therefore look like what we would usually call a paragraph. The naming may be a bit confusing, but since the code that creates and manages superparagraphs has been added on top of the existing code for paragraphs, we considered that for now it would be better to keep the consistency with the previous versions of the tool.

To see how superparagraphs are used when computing the Pad heuristics, functions `superparagraph_average_length` and `average_paragraphs_per_superparagraph` can be seen in `Pad.py`.